In [38]:
#for the scanning
from skimage.filters import threshold_local
import numpy as np
import cv2
import imutils
import os 

#for the cnn
from numpy import loadtxt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [39]:
#Functions linked to scanning

def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")

	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]

	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]

	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect

	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))

	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))

	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")

	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

	# return the warped image
	return warped

def findLargestCountours(cntList, cntWidths):
    newCntList = []
    newCntWidths = []

    # finding 1st largest rectangle
    first_largest_cnt_pos = cntWidths.index(max(cntWidths))

    # adding it in new
    newCntList.append(cntList[first_largest_cnt_pos])
    newCntWidths.append(cntWidths[first_largest_cnt_pos])

    # removing it from old
    cntList.pop(first_largest_cnt_pos)
    cntWidths.pop(first_largest_cnt_pos)

    return newCntList, newCntWidths


def scan(image):
    ratio = image.shape[0] / 500.0
    orig = image.copy()
    image = imutils.resize(image, height = 500)

    
    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
   

    # find the contours in the edged image, keeping only the
    # largest ones, and initialize the screen contour
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)

    screenCntList = []
    scrWidths = []
    
    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        screenCnt = approx

        # look at all contour with 4 edges and add them all in a list
        if len(screenCnt) == 4:
            (X, Y, W, H) = cv2.boundingRect(c)
            screenCntList.append(screenCnt)
            scrWidths.append(W)
    
    if not len(screenCntList) >= 1:  # there is no rectangle found
        print("No rectangle detected in the photo")
    
    
    for cnt in screenCntList:
        image = cv2.drawContours(image, [cnt], -1, (0, 255, 0), 2)
    
    
    #find the largest countour
    screenCntBiggest, scrWidthsBiggest = findLargestCountours(screenCntList, scrWidths)
    
    
    # apply the four point transform to obtain a top-down
    # view of the original image
    warped = four_point_transform(orig, screenCntBiggest[0].reshape(4, 2) * ratio)

    #check if it is a LUS image
    IMAGE_WIDTH=254
    IMAGE_HEIGHT=254
    IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
    #result,prediction = is_LUS_image(warped, IMAGE_SIZE)
    #print(result)
    #print(prediction)
    
    LUS_image = True
    result = True
    
    while not result:
        screenCntBiggest, scrWidthsBiggest = findLargestCountours(screenCntList, scrWidths)  #look for the 2nd biggest rectangle
        warped = four_point_transform(orig, screenCntBiggest[0].reshape(4, 2) * ratio)
        result,prediction = is_LUS_image(warped, IMAGE_SIZE)
        
        if not screenCntList:  #check if list is empty
            print("No LUS image detected")
            LUS_image = False #don't return the scan image but a False boolean
            break
    
    return warped, LUS_image

In [40]:
#Functions linked to CNN

def is_LUS_image(image, IMAGE_SIZE):
    # load model
    model = load_model('/LUS_CNN/model.h5')
    image = adapt_image_for_cnn(image, IMAGE_SIZE)
    predictions = model.predict(image)
    predictions = predictions[0]
    predictions = round(predictions[0],4)
    if predictions < 0.3: 
        predicted_label='YES.  /'+ '  Score = '+ str(predictions)
        result = True
    else: 
        predicted_label='NO   /'+ '  Score = ' + str(predictions)
        result = False 
        
    return result, predicted_label


def adapt_image_for_cnn(img, IMAGE_SIZE):
    img = cv2.resize(img, IMAGE_SIZE)
    img_tensor = img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]
    return img_tensor



In [41]:
#to be modified
directory_scan = 'DIRECTORY OF IMAGE DATASET YOU WANT TO SCAN'

for filename in os.listdir(directory_scan):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image = cv2.imread(os.path.join(directory_scan ,filename))
       
        scanned, LUS_image = scan(image)
        
        if LUS_image:
            #to be modified
            path = 'PATH WHERE YOU WANT TO STORE THE SCANNED IMAGES'
            cv2.imwrite(os.path.join(path ,filename), scanned)
    else:
        continue
    